In [6]:
sc.stop()
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.driver.memory","11g")
         .set("spark.executor.memory","11g")
       )
sc = SparkContext(conf = conf)

In [ ]:
import time, sys, cherrypy, os
from paste.translogger import TransLogger
from app import create_app
from pyspark import SparkContext, SparkConf
 
def init_spark_context():
    # load spark context
    conf = SparkConf().setAppName("movie_recommendation-server")
    # IMPORTANT: pass aditional Python modules to each worker
    sc = SparkContext(conf=conf, pyFiles=['engine.py', 'app.py'])
 
    return sc
 
 
def run_server(app):
 
    # Enable WSGI access logging via Paste
    app_logged = TransLogger(app)
 
    # Mount the WSGI callable object (app) on the root directory
    cherrypy.tree.graft(app_logged, '/')
 
    # Set the configuration of the web server
    cherrypy.config.update({
        'engine.autoreload.on': True,
        'log.screen': True,
        'server.socket_port': 5432,
        'server.socket_host': '0.0.0.0'
    })
 
    # Start the CherryPy WSGI web server
    cherrypy.engine.start()
    cherrypy.engine.block()
 
 
if __name__ == "__main__":
    # Init spark context and load libraries
    #sc = init_spark_context()
    dataset_path = os.path.join('datasets', 'ml-latest')
    app = create_app(sc, dataset_path)
 
    # start web server
    run_server(app)

INFO:engine:Starting up the Recommendation Engine: 
INFO:engine:Loading Ratings data...
INFO:engine:Loading Movies data...
INFO:engine:Counting movie ratings...
INFO:engine:Training the ALS model...
INFO:engine:ALS model built!
[06/Feb/2017:21:00:27] ENGINE Bus STARTING
INFO:cherrypy.error:[06/Feb/2017:21:00:27] ENGINE Bus STARTING
[06/Feb/2017:21:00:27] ENGINE Started monitor thread '_TimeoutMonitor'.
INFO:cherrypy.error:[06/Feb/2017:21:00:27] ENGINE Started monitor thread '_TimeoutMonitor'.
[06/Feb/2017:21:00:27] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[06/Feb/2017:21:00:27] ENGINE Started monitor thread 'Autoreloader'.
[06/Feb/2017:21:00:27] ENGINE Serving on http://0.0.0.0:5432
INFO:cherrypy.error:[06/Feb/2017:21:00:27] ENGINE Serving on http://0.0.0.0:5432
[06/Feb/2017:21:00:27] ENGINE Bus STARTED
INFO:cherrypy.error:[06/Feb/2017:21:00:27] ENGINE Bus STARTED
127.0.0.1 - - [06/Feb/2017:21:02:25 +0000] "GET / HTTP/1.1" 404 233 "-" "curl/7.35.0"
127.0.0.1 - 